# Automatically Convert Step Files to OpenFOAM with an Observer
Now that we have methods which convert our STEP files directly to some openfoam case let us create an observer which will look at some target folder and 
automatically convert a file to an openfoam case if a new file is generated. 

In [1]:
import pathlib as pt
from collections import OrderedDict
import abc

class FileObserver(abc.ABC):

    target_extension = None

    def __init__(
        self, target_folder: pt.Path, event_handler
    ):
        # event handler takes in some target folder to observe for new step files
        self.target_folder = target_folder
        # create a dictionary to keep track of the step files
        self.cases         = OrderedDict()
        # the event_handler is some function which will take in the path to the
        # new file and perform some kind of action
        self.event_handler = event_handler

    def update_case_files(self):
        # scan for files within the folder with the desired extension
        files = list(self.target_folder.glob("*" + self.target_extension))
        # for all items within the files
        for item in files:
            # if the item is not within the keys
            if item not in self.cases.keys():
                # add the case to the case dictionary and set it to false i.e. not processed
                self.cases[item.absolute()] = False

    def run(self):
        while True:
            # update all the case files
            self.update_case_files()
            for key, value in self.cases.items():
                if not value:
                    self.event_handler(key)
                    self.cases[key] = True

class Step(FileObserver):
    target_extension = "STEP"

Now that the necessary classes have been created we can run the observer with the necessary handler

In [2]:
import arterygen as ag 

base_folder = pt.Path("./example_automatic_conversion")
# within the base_folder we show a simple script which can be run using "python convert_step_to_foam.py"
# which uses the code within the artergygen package 
step_folder = base_folder/"step_files"
foam_folder = base_folder/"foam_cases"

assert all([step_folder.exists(), foam_folder.exists()])

# create an event handler for step files
event_handler = ag.watchdog.handlers.STEPToFoamNewtonianSteadyFoam(foam_folder)
# create our observer which will launch the event handler on any updates
observer      = Step(step_folder, event_handler)
observer.run()
# we can stop the program by stopping the notebook.
# note the code for ag.watchdog.handlers.STEPToFoamNewtonianSteadyFoam will check
# as to whether or not foam folder is already generated. If the foam folder already exists the
# program will remain idle

Info: processing d:\Github\IdealBifurcations\docs\example_automatic_conversion\step_files\base_01_inlet_3.60_bif_2.0_outlet_2.4_angle_0.STEP
Server: Pointwise V18.3

Info: Units specified in the file: Millimeters (0.001 meters)

Info: Model Size adjusted to 100

Info: Begin: aniso layer   1: growthRate = 1.000000, height = 1.000000, totalHeight = 1.000000

Info: End  : aniso layer   1: numPts =     34804, numTets =     99621, (0.885000 seconds)

Info: Begin: aniso layer   2: growthRate = 1.100000, height = 1.100000, totalHeight = 2.100000

Info: End  : aniso layer   2: numPts =     51411, numTets =    199242, (0.940000 seconds)

Info: Begin: aniso layer   3: growthRate = 1.100000, height = 1.210000, totalHeight = 3.310000

Info: End  : aniso layer   3: numPts =     67997, numTets =    298756, (0.869000 seconds)

Info: Begin: aniso layer   4: growthRate = 1.100000, height = 1.331000, totalHeight = 4.641000

Info: End  : aniso layer   4: numPts =     84477, numTets =    397634, (0.917000